In [24]:
import requests
import pandas as pd
from pandas import DataFrame 
!pip install geopy
from geopy.extra.rate_limiter import RateLimiter


import matplotlib.pyplot as plt
import zipfile
import numpy as np
import io
import time
from pprint import pprint 
print("Import Successful")
#!wget https://www150.statcan.gc.ca/n1/tbl/csv/10100084-eng.zip
#!unzip 10100084-eng.zip

Import Successful


# Downloading the data

In [25]:



df = pd.read_csv("TorontoListings.csv")


# Data Cleaning

In [26]:
df.describe()

,id,host_id,neighbourhood_group,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,1.874600e+04,1.874600e+04,0.0,18746.000000,18746.000000,18746.000000,18746.000000,18746.000000,14788.000000,18746.000000,18746.000000
mean,2.673976e+07,1.087340e+08,NaN,43.680717,-79.397420,135.468473,10.786568,26.635602,1.214266,4.715246,125.690761
std,1.298332e+07,1.055992e+08,NaN,0.048263,0.064453,263.761517,36.913904,52.418576,1.597121,10.354866,136.844548
min,1.419000e+03,1.565000e+03,NaN,43.586710,-79.634800,11.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,1.634553e+07,2.049040e+07,NaN,43.645170,-79.426310,61.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,2.775978e+07,6.787450e+07,NaN,43.662815,-79.397350,99.000000,2.000000,6.000000,0.600000,1.000000,83.000000
75%,3.853432e+07,1.823192e+08,NaN,43.700153,-79.376860,150.000000,7.000000,27.000000,1.580000,3.000000,252.000000
max,4.578971e+07,3.711952e+08,NaN,43.836900,-79.127810,13137.000000,1125.000000,828.000000,15.440000,88.000000,365.000000


From above, We can see there is a column which is empty, Hence, We drop empty, NAN data from the set. 

In [27]:
df.head(300)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,1419,Beautiful home in amazing area!,1565,Alexandra,NaN,Little Portugal,43.64617,-79.42451,Entire home/apt,469,4,7,2017-12-04,0.11,1,0
1,8077,Downtown Harbourfront Private Room,22795,Kathie & Larry,NaN,Waterfront Communities-The Island,43.64105,-79.37628,Private room,98,180,169,2013-08-27,1.24,2,365
2,12604,Seaton Village Parlour Bedroom,48239,Rona,NaN,Annex,43.66724,-79.41598,Private room,66,1,0,NaN,NaN,1,0
3,23691,Queen Bedroom close to downtown,93825,Yohan & Sarah,NaN,Briar Hill-Belgravia,43.69602,-79.45468,Private room,70,1,217,2019-12-22,1.72,2,240
4,26654,World Class downtown @CN Tower Theatre MTCC ga...,113345,Adela,NaN,Waterfront Communities-The Island,43.64530,-79.38940,Entire home/apt,125,21,40,2020-03-20,0.34,2,295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1294243,Enjoy Toronto Junction.,1025890,Amir,NaN,Junction Area,43.66711,-79.46453,Entire home/apt,80,365,5,2018-08-19,0.06,1,365
296,1299374,Beaches Butterfly Garden Suite,7065012,Jude,NaN,East End-Danforth,43.67828,-79.30960,Entire home/apt,93,7,62,2019-12-31,0.71,2,35
297,1301516,Luxury on Lake 2 Stories Penthouse,6169934,Fahad,NaN,Niagara,43.63531,-79.40055,Entire home/apt,643,30,8,2014-07-02,0.09,2,365
298,1301718,Uber Upper Beach 3 Bedroom Suite,377846,Nick & Nena,NaN,East End-Danforth,43.68345,-79.29434,Entire home/apt,107,7,21,2020-10-17,0.27,10,238


In [28]:

df.dtypes

id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group               float64
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

Since id and host_name, both are giving the same information, We keep one of them and We will do the same thing for name and coordination(lattitude+longitude), We will concatenate them into coulumn location

In [29]:
df['location'] = [', '.join(str(x) for x in y) for y in map(tuple, df[['latitude', 'longitude']].values)]
df.head(20)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,location
0,1419,Beautiful home in amazing area!,1565,Alexandra,NaN,Little Portugal,43.64617,-79.42451,Entire home/apt,469,4,7,2017-12-04,0.11,1,0,"43.64617, -79.42451"
1,8077,Downtown Harbourfront Private Room,22795,Kathie & Larry,NaN,Waterfront Communities-The Island,43.64105,-79.37628,Private room,98,180,169,2013-08-27,1.24,2,365,"43.64105, -79.37628000000001"
2,12604,Seaton Village Parlour Bedroom,48239,Rona,NaN,Annex,43.66724,-79.41598,Private room,66,1,0,NaN,NaN,1,0,"43.66724, -79.41598"
3,23691,Queen Bedroom close to downtown,93825,Yohan & Sarah,NaN,Briar Hill-Belgravia,43.69602,-79.45468,Private room,70,1,217,2019-12-22,1.72,2,240,"43.696020000000004, -79.45468000000001"
4,26654,World Class downtown @CN Tower Theatre MTCC ga...,113345,Adela,NaN,Waterfront Communities-The Island,43.64530,-79.38940,Entire home/apt,125,21,40,2020-03-20,0.34,2,295,"43.6453, -79.3894"
5,27423,Executive Studio Unit- Ideal for One Person,118124,Brent,NaN,Greenwood-Coxwell,43.66890,-79.32592,Entire home/apt,54,120,26,2011-08-30,0.21,1,0,"43.6689, -79.32592"
6,28160,"Luxury, Safety, Affordability For Women Travel...",86838,Rita,NaN,Mount Pleasant West,43.70376,-79.39077,Entire home/apt,50,60,7,2018-10-17,0.11,1,364,"43.703759999999996, -79.39076999999999"
7,30931,Downtown Toronto - Waterview Condo,22795,Kathie & Larry,NaN,Waterfront Communities-The Island,43.64151,-79.37643,Entire home/apt,131,180,1,2010-08-11,0.01,2,365,"43.64151, -79.37643"
8,40456,Downtown 2 Bdr.Apt with King Size Bed and Par...,174063,Denis,NaN,South Parkdale,43.63532,-79.44049,Entire home/apt,100,30,110,2020-03-25,0.88,5,359,"43.63532, -79.44049"
9,41887,Great location,183071,Kyle,NaN,Oakridge,43.69466,-79.28667,Entire home/apt,75,28,82,2019-09-02,1.74,2,342,"43.69466, -79.28667"


In [30]:
df.drop(['neighbourhood_group', 'name', 'host_name'], 1, inplace = True)
df.head()

,id,host_id,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,location
0,1419,1565,Little Portugal,43.64617,-79.42451,Entire home/apt,469,4,7,2017-12-04,0.11,1,0,"43.64617, -79.42451"
1,8077,22795,Waterfront Communities-The Island,43.64105,-79.37628,Private room,98,180,169,2013-08-27,1.24,2,365,"43.64105, -79.37628000000001"
2,12604,48239,Annex,43.66724,-79.41598,Private room,66,1,0,NaN,NaN,1,0,"43.66724, -79.41598"
3,23691,93825,Briar Hill-Belgravia,43.69602,-79.45468,Private room,70,1,217,2019-12-22,1.72,2,240,"43.696020000000004, -79.45468000000001"
4,26654,113345,Waterfront Communities-The Island,43.64530,-79.38940,Entire home/apt,125,21,40,2020-03-20,0.34,2,295,"43.6453, -79.3894"


# taking care of missing data

In [31]:
df = df[['id', 'host_id', 'neighbourhood', 'latitude', 'longitude', 'room_type','minimum_nights', 'number_of_reviews', 'last_review', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365', 'location', 'price']]
cols = list(df.columns.values)
print(cols)
df.head()


['id', 'host_id', 'neighbourhood', 'latitude', 'longitude', 'room_type', 'minimum_nights', 'number_of_reviews', 'last_review', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365', 'location', 'price']


,id,host_id,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,location,price
0,1419,1565,Little Portugal,43.64617,-79.42451,Entire home/apt,4,7,2017-12-04,0.11,1,0,"43.64617, -79.42451",469
1,8077,22795,Waterfront Communities-The Island,43.64105,-79.37628,Private room,180,169,2013-08-27,1.24,2,365,"43.64105, -79.37628000000001",98
2,12604,48239,Annex,43.66724,-79.41598,Private room,1,0,NaN,NaN,1,0,"43.66724, -79.41598",66
3,23691,93825,Briar Hill-Belgravia,43.69602,-79.45468,Private room,1,217,2019-12-22,1.72,2,240,"43.696020000000004, -79.45468000000001",70
4,26654,113345,Waterfront Communities-The Island,43.64530,-79.38940,Entire home/apt,21,40,2020-03-20,0.34,2,295,"43.6453, -79.3894",125


In [32]:
# count the number of missing values for each column
num_missing = (df[['id', 'host_id', 'neighbourhood', 'latitude', 'longitude', 'room_type', 'minimum_nights', 'number_of_reviews', 'last_review', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365', 'location', 'price']] == 0).sum()
# report the results
print(num_missing)

id                                   0
host_id                              0
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
minimum_nights                       0
number_of_reviews                 3958
last_review                          0
reviews_per_month                    0
calculated_host_listings_count       0
availability_365                  6641
location                             0
price                                0
dtype: int64


In [33]:
# replace '0' values with 'nan'
from numpy import nan
df[['number_of_reviews', 'availability_365']] = df[['number_of_reviews', 'availability_365']].replace(0, nan)
# count the number of nan values in each column
print(df.isnull().sum())

id                                   0
host_id                              0
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
minimum_nights                       0
number_of_reviews                 3958
last_review                       3958
reviews_per_month                 3958
calculated_host_listings_count       0
availability_365                  6641
location                             0
price                                0
dtype: int64


In [34]:
#conversion of the 'REF_DATE' from a string to a proper datetime object.
df['last_review'] = pd.to_datetime(df['last_review']) 
# fill missing values with mean column values
df.fillna(df.mean(), inplace=True)
# count the number of NaN values in each column
print(df.isnull().sum())


<ipython-input-34-0c8b10ad8c6d>:4: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df.fillna(df.mean(), inplace=True)


id                                   0
host_id                              0
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
minimum_nights                       0
number_of_reviews                    0
last_review                       3958
reviews_per_month                    0
calculated_host_listings_count       0
availability_365                     0
location                             0
price                                0
dtype: int64


# Encoding categorical Data

In [37]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
X=df.iloc[: , 5:12].values
y=df.iloc[:, 13].values
one_hot = pd.get_dummies(df['room_type'])
print(one_hot)

       Entire home/apt  Hotel room  Private room  Shared room
0                    1           0             0            0
1                    0           0             1            0
2                    0           0             1            0
3                    0           0             1            0
4                    1           0             0            0
...                ...         ...           ...          ...
18741                1           0             0            0
18742                1           0             0            0
18743                1           0             0            0
18744                0           0             1            0
18745                1           0             0            0

[18746 rows x 4 columns]


In [41]:
merged_df = pd.concat([df, one_hot], axis=1)
print(merged_df)

             id    host_id                      neighbourhood  latitude  \
0          1419       1565                    Little Portugal  43.64617   
1          8077      22795  Waterfront Communities-The Island  43.64105   
2         12604      48239                              Annex  43.66724   
3         23691      93825               Briar Hill-Belgravia  43.69602   
4         26654     113345  Waterfront Communities-The Island  43.64530   
...         ...        ...                                ...       ...   
18741  45785334    1023135                     Yonge-Eglinton  43.70703   
18742  45786222  371188170                   Newtonbrook East  43.79075   
18743  45786961  371195218                    Bayview Village  43.76668   
18744  45789233  304805850             Corso Italia-Davenport  43.67330   
18745  45789708  344709977                            Niagara  43.64481   

       longitude        room_type  minimum_nights  number_of_reviews  \
0      -79.42451  Entire ho

In [47]:
#merged_df.drop(['room_type'], 1, inplace = True)
merged_df.columns

Index(['id', 'host_id', 'neighbourhood', 'latitude', 'longitude',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'location', 'price', 'Entire home/apt',
       'Hotel room', 'Private room', 'Shared room'],
      dtype='object')

# Linear regression feature importance 

In [40]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot
# define dataset
X=merged_df.iloc[: , 5:12].values
y=merged_df[:, 13].values
# define the model
model = LinearRegression()
# fit the model
model.fit(X, y)
# get importance
importance = model.coef_
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()

ValueError: could not convert string to float: 'Entire home/apt'

In [ ]:
def getKeys():
    for (key, values) in mydict.items():
        print(key,values)

getKeys()

Tax sheltered deposits at character banks look like they have potential to be interesting. Lets look at that first.

In [ ]:
tax_msk = df[df['VECTOR'] == 'v37229']
tax_msk
#Tax_sheltered = df[column_msk]
#print(Tax_sheltered)

In [ ]:
plt.plot(tax_msk['REF_DATE'], tax_msk['VALUE'])
plt.xlabel("Date")
plt.ylabel("Dollars (MM)")
plt.title("The Amount of TAX sheltered accounts from 1968 to 1994", color="red" ,fontsize=10)

So the amount of money in tax sheltered accounts are clearly raising with time and appears to have some seasonal component. This is not a surprise as the RRSP contribution deadline among other factors are in March. With an extremely sharp increase through 1993. What happened in 1993? The only thing I find of significant importance is Brian Mulrooney's retirement announcement.

# Additional Plotting

Lets now plot all the series in the same plot and see what they look like.

In [ ]:
uniq_series = np.unique(df["VECTOR"])

In [ ]:
plt.rcParams["figure.figsize"] = (18,8)
for key in uniq_series:
    key_mask = df.VECTOR == key
    x = df[key_mask].REF_DATE
    y = df[key_mask].VALUE
    plt.plot(x,y, label = mydict[key])
plt.xlabel('Year')
plt.ylabel('C$ Dollars (MM)')

#box = ax.get_position()
#ax.set_position([box.x0, box.y0 + box.height * 1.1,
#                 box.width, box.height * 2.2])

plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
          fancybox=True, shadow=True, ncol=2, prop={'size': 10})
#plt.legend(fontsize = 'large')

This is looking better, but there is some data from before 1976. What is it?

In [ ]:
df['REF_DATE'].value_counts()

It appears that there are three series in the set that began before 1976. Lets isolate them into a separate plot. First, we identify which vectors are available during that time period:

In [ ]:
veccounts = df[df['REF_DATE'].dt.year == 1969].VECTOR.value_counts()
print(veccounts)

There seem to be three vectors from 1969. Lets extract those into a list.

In [ ]:
veclist = np.unique(df[df['REF_DATE'].dt.year == 1969].VECTOR)
print(veclist)

Lets check what these vectors mean using our dictionary:

In [ ]:
[mydict[vec] for vec in veclist]

The data captured before 1976 are only various forms of deposits between banks. Specifically, these function essentially like chequing accounts that bank A holds with another bank B. Effectively they are like very liquid cash equivalents held at another institution that.

It looks like they are nested subsets of each other. That is the money counted in 'Inter-bank demand deposits' is also contained in 'Inter-bank demand and notice deposits' and so on. Lets confirm by plotting these three.

In [ ]:
for vec in veclist:
    plt_msk = df['VECTOR'] == vec
    plt.plot(df[plt_msk].REF_DATE, df[plt_msk].VALUE, label = mydict[vec])
    plt.title("Inter-bank Demand Deposites Compare to Inter-bank Demand Deposits and Notice Deposites Before 1980")
    #print('Now plotting', mydict[vec])
    #time.sleep(2)
plt.show()

It turns out one of these plots are exactly the same as the other. Lets take a closer look:

In [ ]:
for vec in veclist:
    plt_msk = df['VECTOR'] == vec
    plt.plot(df[plt_msk].REF_DATE, df[plt_msk].VALUE, label = mydict[vec])
    print('Now plotting', mydict[vec])
    plt.show()

Notice that there are only two lines. This is because there are no notice deposits. Hence there are only effectively two plots here:

    'Inter-bank demand deposits', and 'Inter-bank demand and term deposits'.
    
With this in mind. We can now drop these from the dataset and focus on remaining data.

Lets make a new copy of the dataframe and drop the old data.

In [ ]:
df2 = df.copy()

for vec in veclist:
    tmp_ind = df2[df2['VECTOR'] == vec].index
    df2.drop(tmp_ind, axis = 0, inplace = True)



In [ ]:
uniq_series = np.unique(df2["VECTOR"])
plt.rcParams["figure.figsize"] = (18,8)
for key in uniq_series:
    key_mask = df2.VECTOR == key
    x = df2[key_mask].REF_DATE
    y = df2[key_mask].VALUE
    plt.plot(x,y, label = mydict[key])
plt.xlabel('Year')
plt.ylabel('C$ Dollars (MM)')

#box = ax.get_position()
#ax.set_position([box.x0, box.y0 + box.height * 1.1,
#                 box.width, box.height * 2.2])

plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
          fancybox=True, shadow=True, ncol=2, prop={'size': 10})
#plt.legend(fontsize = 'large')

As we can see, now. The vectors corresponding to 'Inter-bank demand deposits and other adjustments to M1', 'Inter-bank notice deposits and other adjustments to M2', and 'Inter-bank term deposits and other adjustments to M3' are extensions of the previous three plots we looked at. In 1976, the Bank of Canada separated these time series so that each one only looked at one thing - demand deposits, notice deposits, and term deposits as opposed to amalgamating them.

Note that since in 1976, there are no notice deposits. We can extent the three above time series back to 1976 by taking the difference of 'Inter-bank demand, notice and term deposits' and 'Inter-bank demand deposits'. For now, let us simply drop the data. As these represent a separate category from the remaining series.

In [ ]:
df3 = df2.copy()

iblist = ['v37226', 'v37227', 'v37228']

for vec in iblist:
    tmp_ind = df3[df3['VECTOR'] == vec].index
    df3.drop(tmp_ind, axis = 0, inplace = True)
    
uniq_series = np.unique(df3["VECTOR"])
plt.rcParams["figure.figsize"] = (18,8)
for key in uniq_series:
    key_mask = df3.VECTOR == key
    x = df3[key_mask].REF_DATE
    y = df3[key_mask].VALUE
    plt.plot(x,y, label = mydict[key])
plt.xlabel('Year')
plt.ylabel('C$ Dollars (MM)')

#box = ax.get_position()
#ax.set_position([box.x0, box.y0 + box.height * 1.1,
#                 box.width, box.height * 2.2])

plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
          fancybox=True, shadow=True, ncol=2, prop={'size': 10})
#plt.legend(fontsize = 'large')

Next, let us group together all the physical currency such as coins and notes. i.e., lets remove the fiat currency from the data frame.

In [ ]:
getKeys()

From here, our list of fiat currencies is:

In [ ]:
fiatList = ['v37232', 'v37231', 'v37230']

In [ ]:
fiat_df = df3[df3['VECTOR'].isin(['v37232', 'v37231', 'v37230'])]
#fiat_df = df3[(df3['VECTOR'] == fiatList[0]) | (df3['VECTOR'] == fiatList[1]) | (df3['VECTOR'] == fiatList[2])]

In [ ]:
for vec in fiatList:
    tmp_ind = fiat_df[fiat_df['VECTOR'] == vec].index
    df3.drop(tmp_ind, axis = 0, inplace = True)
    
uniq_series = np.unique(fiat_df["VECTOR"])
plt.rcParams["figure.figsize"] = (18,8)
for key in uniq_series:
    key_mask = fiat_df.VECTOR == key
    x = fiat_df[key_mask].REF_DATE
    y = fiat_df[key_mask].VALUE
    plt.plot(x,y, label = mydict[key])
plt.xlabel('Year')
plt.ylabel('C$ Dollars (MM)')
plt.title("Distribution Of The Different Currencies Over The Years")

#box = ax.get_position()
#ax.set_position([box.x0, box.y0 + box.height * 1.1,
#                 box.width, box.height * 2.2])

plt.legend(loc='upper right', bbox_to_anchor=(0.5, -0.05),
          fancybox=True, shadow=True, ncol=2, prop={'size': 10})
#plt.legend(fontsize = 'large')

Here, we can see that notes regularly get destroyed and new notes are added to circulation. Furthermore, we notice that coins did not dip in supply. This is not a surprise as coins can last more than 40 years. Furthermore, the Canadian silver 50 cent coin was already rarely used in the 60s with the final print run in large quantities ending in 1967. Furthermore, this plot does not extend far enough for the introduction of the toonie in 1996, or the removal of the penny in 2012.



Erik Chan  11:41 AM
I have a question about the python project if this is a good time






Erik Chan  11:49 AM
I'll just write it out instead:
I have been cleaning and looking at relationships in the data, but I keep finding that time and time again, the main take away I am getting from the data set is that there is a shocking amount of information that is inaccessible necessary to come to various key conclusions.





11:51
For example, a specific kind of deposit started to appear in 1975, and appeared until a legislative change occured in 1983 and data asks the reader to refer to a Bank of Canada Review document from March of 1983 which no longer exists because it is cut off from 1994.





11:52
Without those documents, it isn't very feasible to draw conclusions about some of the most interesting data points. Now my question is as follows: Is that in and of itself a conclusion I can use for the data set? (edited) 






Kristine  1:19 PM
Yes, absolutely!  I think that pointing out the limitations of the data would in fact be very useful.  For the purposes of Friday's exercise, we would also like you to be able to develop a visualization which helps to explain the data succinctly.  ANY graphical representation (think pie chart, graph, etc) counts here.  The use of visuals is a good tool in oral communication, but your bottom line - i.e. that the data is extremely limited - is a perfectly valid analysis.  Sorry for the slow reply.






Erik Chan  1:22 PM
That's very helpful. Thank you for the feedback






Kristine  1:51 PM
You’re welcome.  Feel free to repeat it.